# Extractice QA :  SQuAD

This is a simple notebook to download and store the [SQuAD v1.1](https://arxiv.org/abs/1606.05250) extractive QA dataset. 

We will take it from Huggingface datasets libray (<3) and turn it into one of the two formats that classy is able to parse (i.e. tsv and jsonl).

After creating this dataset you can train your squad model by exectuing the following bash command:
```bash
classy train qa data/qa/squad-v1.1 -n my_firt_squad_run
```

In [ ]:
! pip install datasets

In [ ]:
import json
import os

from datasets import load_dataset
from tqdm.notebook import tqdm

In [ ]:
# here we load the dataset dataset from "datasets"
dataset = load_dataset("squad")

In [ ]:
dataset

In [ ]:
dataset["train"][0]

In [ ]:
# let's create a repositiory that will contains the dataset splits
import os
dir_path = "squad-v1.1"
os.mkdir(dir_path)

In [ ]:
# if you want the output format to be a tab separated file, decomment  the second line
output_format = "jsonl"
# output_format = "tsv"

In [ ]:
from collections import Counter


for k in ['train', 'validation']:

    with open(f'{dir_path}/{k}.{output_format}', 'w') as f:

        for sample in tqdm(dataset[k], desc=f"Processing {k}"):

            squad_id = sample["id"]
            title = sample["title"]
            context = sample["context"]
            question = sample["question"].lstrip()  # suggested by huggingface to remove left-side white spaces
            
            assert len(sample["answers"]["answer_start"]) == len(sample["answers"]["text"])
            
            # save original answer, and encode as start and end only first
            full_answers = sample["answers"]
            answers_text = sample["answers"]["text"][0]
            answers_start = sample["answers"]["answer_start"][0]
            answers_end = answers_start + len(answers_text)
            assert answers_text == context[answers_start: answers_end]
            
            # write
            f.write(json.dumps(
                dict(
                        context=context,
                        question=question,
                        answer_start=answers_start,
                        answer_end=answers_end,
                        squad_id=squad_id,
                        title=title,
                        full_answers=full_answers,
                    )
            ) + "\n")

In [ ]:
! head -5 $dir_path/train.$output_format

In [ ]:
! head -5 $dir_path/validation.$output_format